In [2]:
import base64
import json
import requests
from os import getenv, path
from dotenv import load_dotenv
from pdf2image import convert_from_path
import io
from PIL import Image
from prompts import OCR_PROMPT

load_dotenv()

def encode_file_to_base64(file_path):
    """Convierte un archivo (PDF o imagen) a base64"""
    # Obtener la extensión del archivo
    extension = path.splitext(file_path)[1].lower()
    
    try:
        if extension == '.pdf':
            # Convertir PDF a imagen
            images = convert_from_path(file_path)
            if not images:
                raise Exception("No se pudo extraer ninguna imagen del PDF")
            
            # Usar la primera página
            img_byte_arr = io.BytesIO()
            images[0].save(img_byte_arr, format='PNG')
            img_byte_arr = img_byte_arr.getvalue()
            return base64.b64encode(img_byte_arr).decode('utf-8')
        else:
            # Para archivos de imagen
            with open(file_path, "rb") as file:
                return base64.b64encode(file.read()).decode('utf-8')
    except Exception as e:
        raise Exception(f"Error al codificar el archivo: {str(e)}")
    
def analizar_documento(ruta_archivo):
    """Analiza un documento (PDF o imagen) usando el modelo Qwen"""
    try:
        archivo_base64 = encode_file_to_base64(ruta_archivo)
        
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {getenv('OPENROUTER_API_KEY')}",
                "Content-Type": "application/json",
            },
            data=json.dumps({
                "model": "qwen/qwen2.5-vl-72b-instruct:free",
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": OCR_PROMPT
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{archivo_base64}"
                                }
                            }
                        ]
                    }
                ]
            })
        )
        
        resultado = response.json()
        
        # Manejar errores específicos de la API
        if 'error' in resultado:
            error_code = resultado['error'].get('code')
            error_message = resultado['error'].get('message')
            provider = resultado['error'].get('metadata', {}).get('provider_name', 'desconocido')
            
            if error_code == 429:
                return (f"Error: Demasiadas solicitudes al proveedor {provider}. "
                       "Por favor, espera unos minutos antes de intentar nuevamente.")
            else:
                return f"Error del proveedor {provider}: {error_message} (Código: {error_code})"
        
        # Si no hay error, procesar la respuesta normal
        if 'choices' in resultado:
            return resultado['choices'][0]['message']['content']
        else:
            print("Respuesta inesperada:")
            print(json.dumps(resultado, indent=2))
            return "Error: Formato de respuesta inesperado"
            
    except Exception as e:
        return f"Error al procesar el documento: {str(e)}"

def procesar_resultado_json(json_string):
    """
    Procesa el resultado JSON del análisis del documento y lo convierte en un diccionario Python.
    
    Args:
        json_string (str): String que contiene el JSON a procesar
        
    Returns:
        dict: Diccionario con la información procesada
    """
    try:
        # Eliminar los caracteres de formato markdown si están presentes
        if json_string.startswith("```json"):
            json_string = json_string.replace("```json", "", 1)
        if json_string.endswith("```"):
            json_string = json_string.replace("```", "", 1)
            
        # Limpiar espacios en blanco innecesarios
        json_string = json_string.strip()
        
        # Convertir el string a diccionario
        resultado = json.loads(json_string)
        
        return resultado
    except json.JSONDecodeError as e:
        raise Exception(f"Error al decodificar el JSON: {str(e)}")
    except Exception as e:
        raise Exception(f"Error inesperado al procesar el JSON: {str(e)}")


In [4]:
print(descripcion)

```json
{
    "document": {
        "type": "Bill of Lading",
        "number": "SSOF090406718",
        "date_of_issue": "15/08/2010",
        "date_of_shipment": "20/08/2010"
    },
    "entities": [
        {
            "name": "Shenzhen Ailisheng Trade Co., Ltd.",
            "role": "Shipper",
            "address": "Phoenix Road, Luohu district, Guangdong",
            "city": "Shenzhen city",
            "country": "China",
            "postal_code": "",
            "phone": "086-755-36922075",
            "email": ""
        },
        {
            "name": "Alejead Pc S.A.S",
            "role": "Consignee",
            "address": "Carrera 100 5-39 - Cali - Valle - Colombia",
            "city": "Cali",
            "country": "Colombia",
            "postal_code": "28059",
            "phone": "059-032-4491451",
            "email": "alejead@hotmail.com"
        },
        {
            "name": "Agencia de Aduanas Siacomex Ltda",
            "role": "Customs brokers",
       

In [3]:
# El usuario puede especificar la ruta de su imagen
ruta_documento = "billoflading.pdf"
    
try:
    descripcion = analizar_documento(ruta_documento)
    resultado_procesado = procesar_resultado_json(descripcion)

    print(resultado_procesado)
except FileNotFoundError:
    print("Error: No se pudo encontrar la imagen en la ruta especificada.")
except Exception as e:
    print(f"Error inesperado: {str(e)}")

{'document': {'type': 'Bill of Lading', 'number': 'SSOF090406718', 'date_of_issue': '15/08/2010', 'date_of_shipment': '20/08/2010'}, 'entities': [{'name': 'Shenzhen Ailisheng Trade Co., Ltd.', 'role': 'Shipper', 'address': 'Phoenix Road, Luohu district, Guangdong', 'city': 'Shenzhen city', 'country': 'China', 'postal_code': '', 'phone': '086-755-36922075', 'email': ''}, {'name': 'Alejead Pc S.A.S', 'role': 'Consignee', 'address': 'Carrera 100 5-39 - Cali - Valle - Colombia', 'city': 'Cali', 'country': 'Colombia', 'postal_code': '28059', 'phone': '059-032-4491451', 'email': 'alejead@hotmail.com'}, {'name': 'Agencia de Aduanas Siacomex Ltda', 'role': 'Customs brokers', 'address': 'Calle 2 No. 2°-58', 'city': 'Buenaventura', 'country': 'Colombia', 'postal_code': '', 'phone': '(052) 242 2798', 'email': 'buenaventura@siacomex.com'}, {'name': 'Maersk Line', 'role': 'Shipping company (carrier)', 'address': '', 'city': '', 'country': '', 'postal_code': '', 'phone': '', 'email': ''}], 'individu